In [42]:
import time
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler, PatternMatchingEventHandler
import duckdb
import pathlib
import pandas
from pandas_profiling import ProfileReport
import numpy as np
from scipy import stats

conn_formatted = duckdb.connect(database=str(pathlib.Path().resolve().parent) + '/formatted/my-db.duckdb', read_only=False)
conn_trusted = duckdb.connect(database='my-db.duckdb', read_only=False)

cursor = conn_formatted.cursor()
cursor2 = conn_trusted.cursor()
fetch_query = "SHOW TABLES"
cursor.execute(fetch_query)
print(cursor.fetchall())

[('IQ_part1',), ('IQ_part2',), ('countrycode',), ('student_data_2015',)]


In [43]:
cursor.execute(fetch_query)
tables = cursor.fetchall()
for table in tables:
    print(f"Getting table {table[0]}" )
    df = cursor.execute(f"SELECT * FROM {table[0]}").df()
    name_no_prefix = table[0].split("_")[0]
    
    cursor2.execute("SHOW TABLES")
    existing_tables = [x[0] for x in cursor2.fetchall()]
    # check if table already exists
    if name_no_prefix not in existing_tables:
        print(name_no_prefix)
        print("Creating table")
        cursor2.execute(f"CREATE TABLE IF NOT EXISTS {name_no_prefix} AS SELECT * FROM df")
    else:
        print("Inserting into table")
        cursor2.execute(f"INSERT INTO {name_no_prefix} SELECT * FROM df")

Getting table IQ_part1
IQ
Creating table
Getting table IQ_part2
Inserting into table
Getting table countrycode
countrycode
Creating table
Getting table student_data_2015
student
Creating table


In [44]:
#cursor2.execute(f"SELECT * FROM IQ")
#print(cursor2.fetchall())

In [45]:
cursor2.execute("SHOW TABLES")
cursor2.fetchall()

[('IQ',), ('countrycode',), ('student',)]

In [12]:
def data_quality(df):
    profile = ProfileReport(df, title="Profiling Report")
    profile.to_widgets()
    #profile.to_notebook_iframe()

In [56]:
cursor2.execute(fetch_query)
tables = [x[0] for x in cursor2.fetchall()]

for table in tables:
    print(f"Report for table {table}")
    df = cursor2.execute(f"SELECT * FROM {table}").df()
    data_quality(df)

Report for table IQ


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

Report for table countrycode


...

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

Report for table student


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

In [61]:
cursor2.execute(fetch_query)
tables = [x[0] for x in cursor2.fetchall()]

for table in tables:
    print(f"Removing things for table {table}")
    name_no_prefix = table.split("_")[0]
    df = cursor2.execute(f"SELECT * FROM {table}").df()
    #print(df)
    #drop_numerical_outliers(df)
    #print(df)
    df_clean = df_no_outliers.drop_duplicates()
    # removing previous table
    cursor2.execute(f"DROP TABLE {table}")
    # create new one
    cursor2.execute(f"CREATE TABLE IF NOT EXISTS {name_no_prefix} AS SELECT * FROM df")

Removing things for table IQ
Removing things for table countrycode
Removing things for table student


In [29]:
conn_trusted.close()
conn_formatted.close()